In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import datetime
import uule_grabber
import time
import secrets
import csv
from random import randint
# from random_user_agent.user_agent import UserAgent

# useragent = UserAgent()

chromedriver_autoinstaller.install()

while True:
    typedispley = input('d or m - ')
    if typedispley == 'm' or typedispley == 'd':
        break
        
def getuseragent():
    user_agent = useragent.get_random_user_agent()
    return user_agent

myshop = 'KENZI'
filegeo = 'geo_u.txt'
with open(f'{filegeo}', 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
        geo = list(map(lambda x: x.replace('\n', ''), proxies1))
lich_geo = len(geo)      
filegeo = 'keywords_kenzi.txt'
with open(f'{filegeo}', 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
        keywords = list(map(lambda x: x.replace('\n', ''), proxies1))
        
user_agentd = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
hl = 'ru' #hl=sv          Swedish https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
country = 'ua' #se 	Sweden  
num = '10' # Кількість результатів пошуку
zone = 'com.ua' #'com.ua'
# region = 'Kyiv,Kyiv city,Ukraine'  #'Kyiv,Kyiv city,Ukraine', 'UA'   New York,New York,United States
# uule = uule_grabber.uule(region)
opys = {}
brands = {}
results = {}
lich = 0
lich4 = 0
for region in geo:
    posmyshop = ''
    uule = uule_grabber.uule(region)
    city = region.split(',')[0]
    keyword = secrets.choice(keywords)

    chrome_options = webdriver.ChromeOptions()
#     chrome_options.headless = True
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
#     user_agent = useragent.get_random_user_agent()
#     if (typedispley == 'm'):
#         while True:
#             user_agent = getuseragent()
#             if 'Android' in user_agent:
#                 break
#         mobile_emulation = { "deviceName": "Nexus 5" }
#         chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
#         lichdisp = 2
#     elif (typedispley == 'd'):
#         chrome_options.add_argument("start-maximized")
#         user_agent = user_agentd
#         lichdisp = 4
        
#     chrome_options.add_argument(f"user-agent={user_agent}")

    if (typedispley == 'm'):
        mobile_emulation = { "deviceName": "Nexus 5" }
        chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
        user_agent = 'Mozilla/5.0 (Linux; Android 7.0; Moto G (4) Build/NPJS25.93-14-4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.109 Mobile Safari/537.36'
        lichdisp = 2
    elif (typedispley == 'd'):
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        chrome_options.add_argument("start-maximized")
        lichdisp = 4
        
    chrome_options.add_argument(f"user-agent={user_agent}")

    dt = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

#     print(city, keyword)
    browser = webdriver.Chrome(options=chrome_options)
    if ('і' in keyword or 'чохол' in keyword):
        hl = 'ua'
    else:
        hl = 'ru'
    browser.get(f'https://google.com.ua/search?q={keyword}&pws=0&uule={uule}&hl={hl}&gl={country}')
    time.sleep(10)
    
#     opys = {}
#     brands = {}
    position = '~ Відсутній показ!'
    try:
        if (browser.find_elements(By.XPATH, "//form[@id='captcha-form']")):
            browser.close()
            browser.quit()
            break
    except Exception as e:
        print(f"The error '{e}' occurred")
        pass
    try:
        if (browser.find_elements(By.XPATH, "//div[@class='wywECb']") or browser.find_elements(By.XPATH, "//div[@id='result-stats']") or browser.find_elements(By.XPATH, "//li[@class='LLC6Eb']")):
            browser.save_screenshot(f'screenshots/{dt}_{city}_desk.png')
            if (typedispley == 'm'):
                brend = browser.find_elements(By.XPATH, "//span[@class='vBKJcb']")
            elif (typedispley == 'd'):
                brend = browser.find_elements(By.XPATH, "//div[@class='LbUacb']")
            brends = list(map(lambda x: x.text, brend))
            if (myshop in brends):
                position = '- Присутній в шопінгу'
                lich = lich + 1
                posmyshop = f'- позиція {brends.index(myshop) + 1}'
            if (myshop in brends[:lichdisp]):
                position = '* В першому екрані'
                lich4 = lich4 + 1
#             if (myshop in brends): 
#                 posmyshop = f'- позиція {brends.index(myshop) + 1}'
            result = f'{city} - {keyword} {position} {posmyshop}'
            print(result)
            results[keyword+'_'+city] = f'{position} {posmyshop}'
#             print(f'{city} - {keyword} {position} - позиція {if (len(brends)>0): brends.index(myshop) + 1}')
            brands[keyword+'_'+city] = brends
#             print(brends)
            if (typedispley == 'm'):
                title = browser.find_elements(By.XPATH, "//div[@class='bXPcId pymv4e']")
            elif (typedispley == 'd'):
                title = browser.find_elements(By.XPATH, "//span[@class='pymv4e']")
            titlesf = list(filter(lambda x: len(x.text) > 0, title))
            titles = list(map(lambda x: x.text, titlesf))
#             print(titles)
#             opys[keyword] = [brends, titles]
            opys[keyword+'_'+city] = [list(tup) for tup in zip(brends, titles)]            
    except Exception as e:
        print(f"The error '{e}' occurred")
        pass


    browser.close()
    browser.quit()
    delay = randint(10,25)
    time.sleep(delay) 
#     break
    
def save_csv(typelist, dict):    
    try:
        file = open(f"{dt_csv}_{typelist}_{myshop}.csv", "w", encoding="utf-8")
        # make write object
        writer = csv.writer(file, lineterminator = '\n')
        for key, value in dict.items():
            writer.writerow([key, value])
        file.close()
        print(f"{dt_csv}_{typelist}.csv - saved")
    except Exception as e:
        print(f"The error '{e}' occurred")
        pass

results['in_shopping'] = f'{(lich*100//lich_geo)}%'
results['first_screen'] = f'{(lich4*100//lich_geo)}%'
dt_csv = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")  
save_csv('vydacha', opys)
save_csv('brands', brands)
save_csv('results', results)
print ('Присутній в шопінгу: ',(lich*100//lich_geo),"%")
print ('В першому екрані: ',(lich4*100//lich_geo),"%")

d or m - d
Vinnytsia - чехол на самсунг с8 книжка * В першому екрані - позиція 1
Lutsk - samsung galaxy a02 чехол книжка ~ Відсутній показ! 
Dnipro - чехол книжка на meizu m6 note * В першому екрані - позиція 1
Kryvyi Rih - чехол книжка на айфон 12 - Присутній в шопінгу - позиція 7
Zhytomyr - чехол редми 7 книжка * В першому екрані - позиція 2
Uzhhorod - чехол книжка на самсунг а30с ~ Відсутній показ! 
Zaporizhzhia - huawei p8 lite 2017 чехол книжка * В першому екрані - позиція 1
Ivano-Frankivsk - чехол книжка самсунг а50 - Присутній в шопінгу - позиція 24
Bila Tserkva - купить чехол на самсунг а10 книжка - Присутній в шопінгу - позиція 5
Kyiv - чехол редми 7 книжка * В першому екрані - позиція 2
Kropyvnytskyi - чехол на самсунг а71 книжка - Присутній в шопінгу - позиція 14
Rivne - редми 7а чехол книжка * В першому екрані - позиція 2
Lviv - samsung galaxy a02 чехол книжка * В першому екрані - позиція 4
Mykolaiv - чехол книжка редми 9т ~ Відсутній показ! 
Odessa - чехол книжка на самсун